<a href="https://colab.research.google.com/github/adityasharma10699/Project_Quality_Prediction_Repos/blob/main/News_UMAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install umap-learn

In [4]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objs as go
import plotly.figure_factory as ff
import umap # use 'pip install umap-learn' or 'conda install -c conda-forge umap-learn'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Data Mining/Class work/Week 8 and later/News.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59996 entries, 0 to 59995
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  59996 non-null  object
 1   Summary   59996 non-null  object
dtypes: object(2)
memory usage: 937.6+ KB


In [11]:
df

,Category,Summary,cleaned_summary
0,0,Reuters - A group of technology companies\incl...,reuters group technology company including tex...
1,0,Reuters - America Online on Thursday said it\p...,reuters america online thursday said plan sell...
2,0,Reuters - A group of consumer electronics\make...,reuters group consumer electronics maker said ...
3,0,Reuters - The mystery of what went wrong for t...,reuters mystery went wrong software industry l...
4,0,AP - The Norwegian hacker famed for developing...,ap norwegian hacker famed developing dvd encry...
...,...,...,...
59991,1,"VAL GARDENA, Italy (Reuters) - Max Rauffer be...",val gardena italy reuters max rauffer became f...
59992,1,Red Sox general manager Theo Epstein acknowled...,red sox general manager theo epstein acknowled...
59993,1,The Miami Dolphins will put their courtship of...,miami dolphin put courtship lsu coach nick sab...
59994,1,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,pittsburgh ny giant time p line steelers recor...


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])

In [9]:
def cleaner(summary):
    soup = BeautifulSoup(summary, 'lxml') # removing HTML entities such as ‘&amp’,’&quot’,'&gt'; lxml is the html parser and shoulp be installed using 'pip install lxml'
    souped = soup.get_text()
    re1 = re.sub(r"(#|@|http://|https://|www)\S*", " ", souped) # substituting hashtags, @mentions, urls, etc with whitespace
    re2 = re.sub("[^A-Za-z]+"," ", re1) # substituting any non-alphabetic character that repeats one or more times with whitespace

    """
    For more info on regular expressions visit -
    https://docs.python.org/3/howto/regex.html
    """

    tokens = nltk.word_tokenize(re2)
    lower_case = [t.lower() for t in tokens]

    stop_words = set(stopwords.words('english'))
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))

    wordnet_lemmatizer = WordNetLemmatizer()
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

df['cleaned_summary'] = df.Summary.apply(cleaner)
df = df[df['cleaned_summary'].map(len) > 0] # removing rows with cleaned summaries of length 0
print("Printing top 5 rows of dataframe showing original and cleaned summaries....")
print(df[['Summary','cleaned_summary']].head())
df['cleaned_summary'] = [" ".join(row) for row in df['cleaned_summary'].values] # joining tokens to create strings. TfidfVectorizer does not accept tokens as input
data = df['cleaned_summary']
Y = df['Category'] # target column
tfidf = TfidfVectorizer(min_df=.0005, ngram_range=(1,3)) # min_df=.0005 means that each ngram (unigram, bigram, & trigram) must be present in at least 30 documents for it to be considered as a token (60000*.0005=30). This is a clever way of feature engineering
tfidf.fit(data) # learn vocabulary of entire data
data_tfidf = tfidf.transform(data) # creating tfidf values
print("The created tokens: \n", tfidf.get_feature_names_out())
print("Shape of tfidf matrix: ", data_tfidf.shape)




<ipython-input-9-0a18f4da5990>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(summary, 'lxml') # removing HTML entities such as ‘&amp’,’&quot’,'&gt'; lxml is the html parser and shoulp be installed using 'pip install lxml'


Printing top 5 rows of dataframe showing original and cleaned summaries....
                                             Summary  \
0  Reuters - A group of technology companies\incl...   
1  Reuters - America Online on Thursday said it\p...   
2  Reuters - A group of consumer electronics\make...   
3  Reuters - The mystery of what went wrong for t...   
4  AP - The Norwegian hacker famed for developing...   

                                     cleaned_summary  
0  [reuters, group, technology, company, includin...  
1  [reuters, america, online, thursday, said, pla...  
2  [reuters, group, consumer, electronics, maker,...  
3  [reuters, mystery, went, wrong, software, indu...  
4  [ap, norwegian, hacker, famed, developing, dvd...  
The created tokens: 
 ['aa' 'aapl' 'aaron' ... 'zook' 'zurich' 'zvonareva']
Shape of tfidf matrix:  (59989, 7210)


In [12]:
digits=list(df['Category'])


In [13]:
# Implementing UMAP to visualize dataset
import umap
u = umap.UMAP(n_components = 2, n_neighbors=15, min_dist=0.1)
x_umap = u.fit_transform(data_tfidf)
digits=list(df['Category'])


In [14]:
data = [go.Scatter(x=x_umap[:,0], y=x_umap[:,1], mode='markers',
                    marker = dict(color=df['Category'], colorscale='Rainbow', opacity=0.5),
                                text=[f'digit: {a}' for a in digits],
                                hoverinfo='text')]

layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 700, height = 700,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()
